# <center>Projet Neo4j - PapersWithCode</center>

**<center>Lucien DEROUET - Antoine PURIER</center>**

# 1. Introduction

&emsp;&emsp;Dans le cadre de notre projet neo4j, nous construisons une base de données graphe à partir des metadonnées de publications scientifiques hébergées sur le site https://paperswithcode.com. Des informations supplémentaires sur ces publications sont également récupérées sur d'autres sites via des API et du scraping.

Le but est ensuite de requêter cette base pour répondre à des questions sur le dataset, telles que:
- Quels sont les papiers les plus référencés?
- Quels sont les mots-clés les plus cités?
- Quels sont les auteurs à l'origine du plus grand nombre de publications?
- ...

## Les Données

La base de donnée a été construite à partir de données issues de plusieurs sources. Le point de départ est la plateforme (Papers With Code)[https://paperswithcode.com/], via son API publique. Les papiers retenus seront ensuite recherchés sur la plateforme (Crossref)[https://www.crossref.org/], à partir de son API publique, afin de compléter les informations. Le site (Arxiv)[https://arxiv.org/] possède un système de classification de thématiques scientifiques, cette information est récoltée via leur API. 

D'autre part, dans le cadre d'une base de donnée orientée graphe, les références entre publications scienfiques sont une donnée de grande importance. Les citations sont collectées via l'(api Crossref)[https://api.crossref.org/swagger-ui/index.html#/], mais quand l'information n'est pas disponible, le programme tente de la récupérer en scrappant le site d'une des principales revues scientifique : (IEEE)[https://ieeexplore.ieee.org/Xplore/home.jsp].

Comme la collecte de données se fait via des requètes HTTP, un soin particulier a été apporté lors de l'implémentation, via l'usage de l'asynchronisme ((asyncio)[https://docs.python.org/fr/3/library/asyncio.html]), mais surtout par la création d'un algorithme cherchant à obtenir le plus d'information possibles en limitant le nombre de requêtes.

Pour un papier ayant $r$ références, $r + 5$ requêtes suffisent, dans le pire des cas, à obtenir les informations générales du papier, la liste des identifiants de ses références, les informations générales de chaque références, et l'indentifiant des références des références (si obtenue en une seule requête). 
L'api de Papers With Code renvoie les métadonnées de publications par blocs de $l$ papiers, un bloc est alors traité en $l * ( r + 5 )$ requêtes, ce qui correspond à une compléxité linéaire en terme de requêtes. L'implémentation fixe une valeur maximale $r = 50$

Ci-dessous se trouve un schéma résumant l'algorithme.

NB : Le DOI est un identifiant unique associé à une publication scientifique.
![diagramme algo](./diag.png)


### Les métadonnées collectées sont les suivantes :
- le titre
- les auteurs
    - le nom
    - l'organisation d'appartenance (université, entreprise etc.)
- la date de publication
- la catégorie arxiv
- la conférence à laquelle le papier a été présenté
    - nom de la conférence
    - lieu de la conférence
    - date
- langue du papier
- liste des mots clés
- DOI
- liste des références
    - DOI de chaque référence
    
    
### Ces métadonnées sont regroupées sous le format JSON, un exemple :
```json
{
    "url_doi": "http://dx.doi.org/10.1016/j.patcog.2021.108439",
    "title": "3D pose estimation and future motion prediction from 2D images",
    "authors": [
        {
            "name": "Ji Yang",
            "organisation": []
        },
        {
            "name": "Li Cheng",
            "organisation": []
        }
    ],
    "date": "2021-11-26",
    "arxiv_category": "cs.CV",
    "conference": {
        "name": "CVPR",
        "location": "NaN",
        "date": "NaN"
    },
    "language": "NaN",
    "publisher": "Elsevier BV",
    "key_words": [
        "Artificial Intelligence",
        "Computer Vision and Pattern Recognition",
        "Signal Processing",
        "Software"
    ],
    "references": [
        "10.1007/978-3-030-01267-0_24",
        "10.1016/j.patcog.2019.05.026",
        "10.1109/cvpr.2017.644",
        "10.1109/cvpr46437.2021.01584",
        "10.1109/cvpr.2018.00551"
    ],
    "doi": "10.1016/j.patcog.2021.108439"
}
```

### Résultat de l'acquisition :

Au total, plus de 18 000 publications et 300 000 références ont été récoltés, en faisant tourner le script environ 5 heures. La vitesse dépend en grande partie de la disponibilité de l'API de Crossref, qui est fortement solicitée.

Le code ayant servi à récupérer et à construire le dataset se trouve en annexe dans le dossier de rendu.

Dans la prochaine section, nous nous connectons à Neo4j et nous créons la base de donnée.

# 2. Création de la base

## Connexion à Neo4j

Nous commençons par vérifier la version de neo4j installée sur la machine.

In [2]:
from neo4j import __version__ as neo4j_version
print(neo4j_version)

4.4.1


Nous créons ensuite une classe pour la connexion à neo4j.

In [10]:
from neo4j import GraphDatabase

class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

Puis nous instantions une connexion.

In [132]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="lucien")

Nous créons maintenant la base de données graphe `paperswithcode`.

In [18]:
conn.query("CREATE OR REPLACE DATABASE paperswithcode")

[]

Puis nous créons les noeuds et les relations du graphe à partir du fichier json.

Nous créons 6 noeuds:
- `Paper`
- `Author`
- `Publisher`
- `Organisation`
- `Conference`
- `Keyword`

Et nous relions ces noeuds à l'aide de 6 relations:
- Un auteur a écrit un papier : `WROTE`
- Un papier a été présenté à une conférence : `PRESENTED_AT`
- Un éditeur a publié un papier : `PUBLISHED`
- Un auteur est affilié à une organisation (université, entreprise, ...) : `AFFILIATED_TO`
- Un papier parle d'un sujet (évoque un mot clé) : `TALKS_ABOUT`
- Un papier fait référence à un autre papier : `REFERS_TO`

In [19]:
# Import du module `time` pour chronométrer nos requêtes
import time

# Fichier à charger (à placer dans le dossier `Import`)
file = "dataset_18K_v7.json"

### Création des noeuds `Paper`

In [20]:
query_string = '''
CREATE CONSTRAINT ON (p:Paper) ASSERT p.doi IS UNIQUE
'''
conn.query(query_string, db='paperswithcode')

[]

In [21]:
query_string = '''
CREATE CONSTRAINT ON (p:Paper) ASSERT exists(p.doi)
'''
conn.query(query_string, db='paperswithcode')

[]

In [22]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"MERGE (p:Paper {{title: paper.title}})
SET
    p.doi = paper.doi,
    p.language = paper.language,
    p.arxiv_category = paper.arxiv_category,
    p.publication_date = paper.date,
    p.references = paper.references",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

88.44607830047607


### Création des noeuds `Author`

In [23]:
query_string = '''
CREATE CONSTRAINT ON (a:Author) ASSERT a.name IS UNIQUE
'''
conn.query(query_string, db='paperswithcode')

[]

In [24]:
query_string = '''
CREATE CONSTRAINT ON (a:Author) ASSERT exists(a.name)
'''
conn.query(query_string, db='paperswithcode')

[]

In [25]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"WITH paper.authors as authors
UNWIND
    CASE
        WHEN authors = [] THEN [null]
        ELSE authors
    END AS aut
WITH aut
WHERE aut IS NOT NULL
MERGE (a:Author {{name: aut.name}})",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

6.913482189178467


### Création des noeuds `Organisation`

In [26]:
query_string = '''
CREATE CONSTRAINT ON (o:Organisation) ASSERT o.name IS UNIQUE
'''
conn.query(query_string, db='paperswithcode')

[]

In [27]:
query_string = '''
CREATE CONSTRAINT ON (o:Organisation) ASSERT exists(o.name)
'''
conn.query(query_string, db='paperswithcode')

[]

In [28]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"WITH paper.authors as authors
UNWIND authors as aut
UNWIND
    CASE
        WHEN aut.organisation = [] THEN [null]
        ELSE aut.organisation
    END AS org
WITH org
WHERE org IS NOT NULL
MERGE (o:Organisation {{name: org.name}})",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

4.291425943374634


### Création des noeuds `Keyword`

In [29]:
query_string = '''
CREATE CONSTRAINT ON (k:Keyword) ASSERT k.name IS UNIQUE
'''
conn.query(query_string, db='paperswithcode')

[]

In [30]:
query_string = '''
CREATE CONSTRAINT ON (k:Keyword) ASSERT exists(k.name)
'''
conn.query(query_string, db='paperswithcode')

[]

In [31]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"WITH paper.key_words as keywords
UNWIND
    CASE
        WHEN keywords = [] THEN [null]
        ELSE keywords
    END AS key
WITH key 
WHERE key IS NOT NULL
MERGE (k:Keyword {{name: key}})",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

2.723806142807007


### Création des noeuds `Publisher`

In [32]:
query_string = '''
CREATE CONSTRAINT ON (pub:Publisher) ASSERT pub.name IS UNIQUE
'''
conn.query(query_string, db='paperswithcode')

[]

In [33]:
query_string = '''
CREATE CONSTRAINT ON (pub:Publisher) ASSERT exists(pub.name)
'''
conn.query(query_string, db='paperswithcode')

[]

In [34]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"MERGE (pub:Publisher {{name: paper.publisher}})",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

2.5937485694885254


### Création des noeuds `Conference`

In [35]:
query_string = '''
CREATE CONSTRAINT ON (c:Conference) ASSERT c.name IS UNIQUE
'''
conn.query(query_string, db='paperswithcode')

[]

In [36]:
query_string = '''
CREATE CONSTRAINT ON (c:Conference) ASSERT exists(c.name)
'''
conn.query(query_string, db='paperswithcode')

[]

In [37]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"WITH paper.conference as conference
MERGE (c:Conference {{name: conference.name}})
SET
    c.date = conference.date,
    c.location = conference.location",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

2.931542158126831


### Création des relations entre les noeuds `Paper` et `Author`

In [38]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"UNWIND paper.authors as aut
MATCH (p:Paper {{doi: paper.doi}}), (a:Author {{name: aut.name}})
MERGE (a)-[:WROTE]->(p)",
{{batchSize:100, parallel:false}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

17.361862421035767


### Création des relations entre les noeuds `Paper` et `Keyword`

In [39]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"UNWIND
    CASE
        WHEN paper.key_words = [] THEN [null]
        ELSE paper.key_words
    END AS key
MATCH (p:Paper {{doi: paper.doi}}), (k:Keyword {{name: key}})
MERGE (p)-[:TALKS_ABOUT]->(k)",
{{batchSize:100, parallel:false}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

16.576802730560303


### Création des relations entre les noeuds `Paper` et `Publisher`

In [40]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"MATCH (p:Paper {{doi: paper.doi}}), (pub:Publisher {{name: paper.publisher}})
MERGE (pub)-[:PUBLISHED]->(p)",
{{batchSize:100, parallel:false}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

12.878684520721436


### Création des relations entre `Paper` et `Conference`

In [41]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"WITH paper, paper.conference as conference
MATCH (p:Paper {{doi: paper.doi}}), (c:Conference {{name: conference.name}})
MERGE (p)-[:PRESENTED_AT]->(c)",
{{batchSize:100, parallel:false}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

13.064643144607544


### Création des relations entre `Author` et `Organisation`

In [42]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"UNWIND paper.authors as aut
UNWIND
    CASE
        WHEN aut.organisation = [] THEN [null]
        ELSE aut.organisation
    END AS org
MATCH (a:Author {{name: aut.name}}), (o:Organisation {{name: org.name}})
MERGE (a)-[:AFFILIATED_TO]->(o)",
{{batchSize:100, parallel:false}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

10.738734006881714


### Création des relations entre `Paper` et `References`

In [43]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"MATCH (p:Paper {{doi: paper.doi}}), (r:Paper)
WHERE r.doi IN p.references
CREATE (p)-[:REFERS_TO]->(r)",
{{batchSize:100, parallel:false}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

19.470998525619507


## Requête sur la Base

Maintenant que la base est créée et notre graphe modélisé, nous la requêtons.

In [49]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

### Requête 1: Combien y-a-t-il de noeuds distincts pour chaque label?

In [172]:
labels = ["Paper", "Author", "Publisher", "Organisation", "Conference", "Keyword"]

d = {"Nombre de Noeuds": pd.NA}
df_res = pd.DataFrame(data=d, index=labels)

for label in labels:
    query_string = f'''
    MATCH (p:{label})
    RETURN count(*)
    '''
    res = [dict(_) for _ in conn.query(query_string, db='paperswithcode')]
    df_res.loc[label, "Nombre de Noeuds"] = res[0]["count(*)"]
    
display(df_res)   

,Nombre de Noeuds
Paper,18300
Author,19426
Publisher,332
Organisation,4637
Conference,34
Keyword,270


### Requête 2: Combien y-a-t-il de relations de chaque type?

In [183]:
relationship_types = ["WROTE", "TALKS_ABOUT", "REFERS_TO", "PRESENTED_AT", "PUBLISHED", "AFFILIATED_TO"]

d = {"Nombre de Relations": pd.NA}
df_res = pd.DataFrame(data=d, index=relationship_types)

for relationship_type in relationship_types:
    query_string = f'''
    MATCH ()-[rel:{relationship_type}]->()
    RETURN count(rel)
    '''
    res = [dict(_) for _ in conn.query(query_string, db='paperswithcode')]
    df_res.loc[relationship_type, "Nombre de Relations"] = res[0]["count(rel)"]
    
display(df_res)  

,Nombre de Relations
WROTE,36067
TALKS_ABOUT,28917
REFERS_TO,37735
PRESENTED_AT,18300
PUBLISHED,18284
AFFILIATED_TO,3982


### Requête 3: Quels sont les papiers les plus référencés?

In [229]:
query_string = '''
MATCH (p:Paper)-[rel:REFERS_TO]->(r:Paper)
RETURN r.title, count(rel) as count
ORDER BY count desc
LIMIT 5
'''

df_res = pd.DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res = df_res.rename(columns={"r.title":"Titre du Papier", "count":"Nombre de Référencements"})
df_res

,Titre du Papier,Nombre de Référencements
0,"The relationship between infrared, optical, and ultraviolet extinction",105
1,"Electric-magnetic duality, monopole condensation, and confinement in N=2 supersymmetric Yang-Mills theory",86
2,emcee: The MCMC Hammer,86
3,SExtractor: Software for source extraction,81
4,Stellar population synthesis at the resolution of 2003,79


### Requête 4: Quelles sont les organisations les plus représentées dans le dataset?

In [230]:
query_string = '''
MATCH (o:Organisation)
RETURN o.name, count(o) as count
ORDER BY count desc
LIMIT 5
'''

df_res = pd.DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res = df_res.rename(columns={"o.name":"Organisation", "count":"Auteurs Affiliés"})
df_res

,Organisation,Auteurs Affiliés
0,*Center for Bits and Atoms and,1
1,"*Department of Biological Sciences, Stanford University, Stanford, CA 94305;",1
2,"*Department of Molecular, Cellular, and Developmental Biology, Yale University, New Haven, CT 06520;",1
3,"*Department of Physics, University of Illinois at Chicago, Chicago, IL 60607;",1
4,"**Departments of Human Genetics, and Psychiatry and Behavioral Sciences, Emory University School of Medicine, Atlanta, GA 30322;",1


### Requête 5: Quels sont les mots clé les plus cités?

In [231]:
query_string = '''
MATCH (p:Paper)-[rel:TALKS_ABOUT]->(k:Keyword)
RETURN k.name, count(rel) as count
ORDER BY count desc
LIMIT 5
'''

df_res = pd.DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res = df_res.rename(columns={"k.name":"Mot-clé", "count":"Nombre de Citations"})
df_res

,Mot-clé,Nombre de Citations
0,Astronomy and Astrophysics,3366
1,Space and Planetary Science,3276
2,Nuclear and High Energy Physics,1930
3,General Physics and Astronomy,1375
4,Electrical and Electronic Engineering,1013


### Requête 6: Quel est le nombre de papiers publiés par publishers?

In [232]:
query_string = '''
MATCH (pub:Publisher)-[rel:PUBLISHED]->(p:Paper)
RETURN pub.name, count(rel) as count
ORDER BY count desc
LIMIT 5
'''

df_res = pd.DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res = df_res.rename(columns={"pub.name":"Éditeur", "count":"Nombre de Publications"})
df_res

,Éditeur,Nombre de Publications
0,Elsevier BV,2288
1,Springer Science and Business Media LLC,2262
2,IEEE,2132
3,American Physical Society (APS),1653
4,American Astronomical Society,1633


### Requête 7: Quels sont les auteurs à l'origine du plus grand nombre de publications?

In [50]:
query_string = '''
MATCH (p:Paper)-[rel1:TALKS_ABOUT]->(k:Keyword)
WHERE k.name = "Artificial Intelligence"
WITH p
MATCH (a:Author)-[rel2:WROTE]->(p:Paper)
RETURN a.name, count(p) as count
ORDER BY count desc
LIMIT 5
'''
df_res = pd.DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res = df_res.rename(columns={"a.name":"Nom de l'Auteur", "count":"Nombre de Papiers Publiés"})
df_res


[<Record a.name='Chunhua Shen' count=6>, <Record a.name='Marco Saerens' count=5>, <Record a.name='Mohammed Bennamoun' count=4>, <Record a.name='Roland Siegwart' count=3>, <Record a.name='Min Lu' count=3>]


,Nom de l'Auteur,Nombre de Papiers Publiés
0,Chunhua Shen,6
1,Marco Saerens,5
2,Mohammed Bennamoun,4
3,Roland Siegwart,3
4,Min Lu,3


### Requête 8: Quels sont les papiers les plus cités par mots clés ?



In [129]:
query_string = '''
 MATCH (p1: Paper)-[r:REFERS_TO]-(p2: Paper)-[r2: TALKS_ABOUT]-(kw: Keyword)
 RETURN kw.name, p2.title, count(p1) AS c
 ORDER BY c DESC LIMIT 5

'''
df_res = pd.DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res = df_res.rename(columns={"kw.name":"mot cle", "c":"Nombre de citation", "p2.title": "papier le plus cite"})
df_res

,mot cle,papier le plus cite,Nombre de citation
0,Space and Planetary Science,"The relationship between infrared, optical, and ultraviolet extinction",105
1,Astronomy and Astrophysics,"The relationship between infrared, optical, and ultraviolet extinction",105
2,Astronomy and Astrophysics,emcee: The MCMC Hammer,87
3,Nuclear and High Energy Physics,"Electric-magnetic duality, monopole condensation, and confinement in N=2 supersymmetric Yang-Mills theory",87
4,Space and Planetary Science,emcee: The MCMC Hammer,87


### Requête 9 Quel est le 


In [134]:
query_string = '''
MATCH (p:Paper)-[rel1:TALKS_ABOUT]->(k:Keyword)
WITH p, k
MATCH (pub:Publisher)-[rel2:PUBLISHED]->(p)
WITH pub, p, k
RETURN pub.name, k.name, count(k)
'''
df_res = pd.DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res = df_res.rename(columns={"pub.name":"Éditeur" , "k.name":"Mot-Clé", "count(k)":"Nombre de Papiers"})
df_res

,Éditeur,Mot-Clé,Nombre de Papiers
0,AI Access Foundation,Artificial Intelligence,3
1,AIP Publishing,Physics and Astronomy (miscellaneous),65
2,AIP Publishing,General Physics and Astronomy,98
3,AIP Publishing,Physical and Theoretical Chemistry,40
4,AIP Publishing,General Chemistry,3
5,AIP Publishing,General Engineering,11
6,AIP Publishing,Fluid Flow and Transfer Processes,23
7,AIP Publishing,Condensed Matter Physics,31
8,AIP Publishing,Mechanical Engineering,21
9,AIP Publishing,Computational Mechanics,22


### Requête 10 : Quelle est la date de publication du tout premier papier pour chaque mot clé ?

In [115]:
query_string = '''
 MATCH (kw: Keyword)
 WITH DISTINCT kw.name as kw_name
 MATCH (p: Paper)-[:TALKS_ABOUT]-(kw: Keyword {name: kw_name})
 WHERE p.publication_date <> "None"
 WITH kw_name as name , p.publication_date as date, p
 ORDER BY date
 RETURN date, name, p.title
 LIMIT 5
'''
df_res = pd.DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res = df_res.rename(columns={"date.name":"publication date", "name.name":"mot cle", "p.title": "papier"})

df_res

,date,name,papier
0,2007-09-21,General Physics and Astronomy,Bayesian Approach to Network Modularity
1,2010-07-17,Astronomy and Astrophysics,3D photometric cosmic shear
2,2010-07-17,Space and Planetary Science,3D photometric cosmic shear
3,2010-12-01,Software,A Bayesian approach to abrupt concept drift
4,2010-12-01,Information Systems and Management,A Bayesian approach to abrupt concept drift


## Fermeture de la Connexion

In [135]:
conn.close()